In [22]:
import pandas as pd
import numpy as np
import sklearn.model_selection
import sklearn.linear_model
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
df = pd.read_csv("Food survey.csv")
df.head()

dishes = df.iloc[:, 1:]

headers = dishes.columns.values

In [24]:
dish_names = headers.copy()
print(dish_names[0])

Patta Gobi


In [25]:
new_headers = np.arange(0, 16)

dishes.columns = new_headers

In [26]:
users = {}
for i in range(36):
    users[i] = str(i)

In [27]:
users

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '10',
 11: '11',
 12: '12',
 13: '13',
 14: '14',
 15: '15',
 16: '16',
 17: '17',
 18: '18',
 19: '19',
 20: '20',
 21: '21',
 22: '22',
 23: '23',
 24: '24',
 25: '25',
 26: '26',
 27: '27',
 28: '28',
 29: '29',
 30: '30',
 31: '31',
 32: '32',
 33: '33',
 34: '34',
 35: '35'}

In [28]:
def item_similarity(interaction_matrix):
    # Transpose the interaction matrix to have items as rows and users as columns
    item_interaction_matrix = interaction_matrix.T
    
    # Calculate cosine similarity between items
    similarity_matrix = cosine_similarity(item_interaction_matrix, dense_output=False)
    
    return similarity_matrix

In [29]:
def select_neighborhood(similarity_matrix, item_id, neighborhood_size):
    # Get the similarity scores for the given item
    item_similarity_scores = similarity_matrix[item_id]
    
    # Sort the items based on similarity scores (excluding the item itself)
    sorted_indices = np.argsort(item_similarity_scores)[::]
    
    # Select the top neighborhood_size similar items (excluding the item itself)
    neighborhood = sorted_indices[1:neighborhood_size+1]
    
    return neighborhood

In [30]:
def update_data(user_id, selected_dish):
    dish_similarity = item_similarity(dishes)
    hood = select_neighborhood(dish_similarity, selected_dish, 5)
    for i in hood:
        if(dish_similarity[user_id][i] < 0.7):
            dish_similarity[user_id][i] += 0.2
            dishes.iloc[user_id][i] += 0.2
        else:
            dish_similarity[user_id][i] -= 0.5
            dishes.iloc[user_id][i] -= 0.5

In [31]:
x = np.array(df.iloc[:, 1:])
y = np.array(df.iloc[:, 0]).reshape(-1,1)

In [32]:
model = sklearn.linear_model.LinearRegression().fit(y, x)

In [33]:
model.predict([[37]])

array([[2.90595238, 3.01111111, 2.84619048, 3.47111111, 2.85492063,
        3.14888889, 2.8968254 , 2.73079365, 2.56504762, 2.42825397,
        2.45047619, 2.60539683, 3.36142857, 3.52031746, 2.50444444,
        2.506     ]])

In [34]:
def add_user(name):
    global dishes
    dict = {}
    new_data = model.predict([[len(dishes)]])

    for i in range(len(new_data[0])):
        dict[i] = new_data[0][i]
    
    print(dict)

    #df2 = pd.DataFrame(dict)

    dishes = dishes._append(dict, ignore_index=True)

    users[len(dishes)] = name

In [35]:
add_user("gersde")

{0: 2.927477477477477, 1: 3.0057057057057053, 2: 2.8837837837837834, 3: 3.496696696696697, 4: 2.883183183183183, 5: 3.183483483483484, 6: 2.921921921921922, 7: 2.75015015015015, 8: 2.5936036036036034, 9: 2.455255255255255, 10: 2.4720720720720717, 11: 2.6300300300300297, 12: 3.3824324324324326, 13: 3.5420420420420418, 14: 2.5444444444444447, 15: 2.5502252252252244}


In [36]:
users

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '10',
 11: '11',
 12: '12',
 13: '13',
 14: '14',
 15: '15',
 16: '16',
 17: '17',
 18: '18',
 19: '19',
 20: '20',
 21: '21',
 22: '22',
 23: '23',
 24: '24',
 25: '25',
 26: '26',
 27: '27',
 28: '28',
 29: '29',
 30: '30',
 31: '31',
 32: '32',
 33: '33',
 34: '34',
 35: '35',
 37: 'gersde'}

In [37]:
def validate_user(id):
    if id in dishes.index:
        return True
    else:
        return False

In [38]:
def getList(id):
    recc = {}
    for i in range(16):
        recc[i] = dishes.iloc[2][i]

    recc = sorted(recc.items(), key = lambda item:item[1])
    return recc

In [39]:
a = getList(2)
b = a[15:10:-1]
print(b)

print(b[1][1])

[(4, 5.0), (3, 5.0), (14, 4.8), (13, 4.8), (9, 4.8)]
5.0


In [44]:
def interact(id):
    print("What would you like to eat today?")
    for i in range(0, len(dish_names)):
        print(i, dish_names[i])

    while (True):
        print("\nEnter your choice, enter 99 to exit, or enter 111 for recommendations.")
        selection = int(input())

        if selection==99:
            break
        elif selection==111:
            print("Perhaps you might enjoy: ")
            recc = getList(id)[15:10:-1]
            for i in recc:
                print(dish_names[i[0]])
            print("Enter your selection: ")
            d = int(input())
            update_data(id, d)

        elif (dishes.iloc[id][selection] > 3):
            print("\nGreat choice, enjoy your meal.\n")
            update_data(id, selection)

In [41]:
dishes.loc[2]

0     4.20
1     1.00
2     4.20
3     5.00
4     5.00
5     4.80
6     4.60
7     4.00
8     4.67
9     4.80
10    4.40
11    4.60
12    4.60
13    4.80
14    4.80
15    4.60
Name: 2, dtype: float64

In [45]:
#Ask the user for their ID
id = int(input("Enter your ID:"))

if not validate_user(id):
    print(dishes.index)
    print("User doesn't exist, making account...")
    print("Please enter your username: ")
    name = input()
    add_user(name)
    print("User ID: ", len(dishes))
else:
    print("Welcome, ", users[id])
    interact(id)

Welcome,  2
What would you like to eat today?
0 Patta Gobi
1 Phool Gobi
2 Bhindi
3 Aloo Sabzi
4 Rajma
5 Paneer
6 Chole
7 Baingan
8 Shimla Mirch
9 Sev Tamatar
10 Malai Pyaaz
11 Mix Veg
12 Any Dal
13 Aloo Paratha
14 Pyaaz Paratha
15 Paneer Paratha

Enter your choice, enter 99 to exit, or enter 111 for recommendations.
Perhaps you might enjoy: 
Rajma
Aloo Sabzi
Pyaaz Paratha
Aloo Paratha
Paneer
Enter your selection: 

Enter your choice, enter 99 to exit, or enter 111 for recommendations.
Perhaps you might enjoy: 
Rajma
Aloo Sabzi
Aloo Paratha
Paneer
Shimla Mirch
Enter your selection: 

Enter your choice, enter 99 to exit, or enter 111 for recommendations.
Perhaps you might enjoy: 
Aloo Sabzi
Aloo Paratha
Shimla Mirch
Rajma
Malai Pyaaz
Enter your selection: 


IndexError: index 99 is out of bounds for axis 0 with size 16